In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# train data
# train_data = joblib.load('./train_data/train_data.lz4') # 一共234列
# data_label = pd.read_csv('../../preprocess_data/train_y_33465.csv',usecols=['label'])
# x_train = train_data.fillna(-1).values
# y_train = data_label.values.ravel()
# valid data
valid_data = joblib.load('./valid_data/valid_data.lz4')
x_valid = valid_data.fillna(-1).values

In [3]:
# train
import lightgbm as lgb
parameters = {
    'boost':'gbdt',
    'num_leaves':120, 
    'scale_pos_weight':float(len(y_train)-np.sum(y_train))/float(np.sum(y_train)),
    'max_depth':-1,
    'learning_rate':.05,
    'max_bin':200,
    'min_data_in_leaf' : 100,
    'objective':'binary',
    'metric':'auc',
    'num_threads':18
}
lgb_train = lgb.Dataset(x_train, y_train)
lgb_model = lgb.train(parameters,lgb_train,num_boost_round=200,verbose_eval=1)

In [4]:
joblib.dump(lgb_model,'./model/lgb_model1')

['./model/lgb_model1']

In [3]:
# predict
valid_id = pd.read_csv('../../preprocess_data_new/valid_date.csv',usecols=['id']).values.ravel()
model = joblib.load('./model/lgb_model')
def predict_prob(model,x_test,test_id):
    pred = pd.DataFrame()
    pred['id'] = test_id
    pred['prob'] = model.predict(x_test)
    return pred
    

In [4]:
pred = predict_prob(model,x_valid,valid_id)

In [5]:
pred.to_csv('./preds/lgb_pred.txt',index=False)

In [8]:
# AUC:0.80971